In [1]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))
from flash_attn import flash_attn_func  # или flash_attn_qkvpacked_func и т.д.

print("flash_attn установлен успешно")

device = "cuda" if torch.cuda.is_available() else "cpu"

True
NVIDIA GeForce RTX 4090
flash_attn установлен успешно


In [7]:
from faster_whisper import WhisperModel

model = WhisperModel("large-v3", device="cuda", compute_type="float16")  # или "int8" для экономии VRAM

segments, info = model.transcribe(
    "../data/audio/julia/audio_2026-02-03_21-33-26.ogg",
    language="ru",
    vad_filter=True,          # убирает молчание
    beam_size=5
)

text = ".".join([s.text for s in segments])
print("Detected language:", info.language)
print("Текст:", text)

Detected language: ru
Текст:  Ой, кстати, из забавного, я не умею готовить пюре. Оно у меня всегда получается отвратительным. Поэтому я его не готовлю и доверяю всем. Вот кому угодно, лишь бы не я это делала. Это вот что-то волшебное. Все что угодно могу, но пюре. Еще пару блюд, которые вот простейшие. У меня не получается, а, у меня омлет не получается. Звучит, конечно, странно. Но то ли он пригорает. То ли еще что-то, то ли еще. И, короче, всегда у меня неудачные омлеты, пюре. Поэтому я подумала. Потом как-нибудь научусь. Пока буду доверять всем подряд. У меня так есть. История с подругой. С подругой, с ее мамой. Мы дружим с Машей с пяти лет. И она живет прямо возле школы. И постоянно перед школой я к ней заходила. У меня была музыкалка. Черчение, и потом после музыкалки. Черчение, я шла к Маше на часа два. И мы обычно завтракали у нее. И вот ее мать готовит. Просто отвратительно. Вот все, что только можно, она готовит отвратительно. Она еще. Какой-то сумасшедший экспериментатор. У н

In [ ]:
from transformers import pipeline

pipe = pipeline(
    "automatic-speech-recognition",
    model="distil-whisper/distil-large-v3",
    device=0,  # cuda:0
    torch_dtype="float16"
)

result = pipe("audio/Vasya/audio_2026-01-30_17-49-34.ogg", return_timestamps=True)
print(result["text"])

In [2]:
from TTS.api import TTS

# Загружаем XTTS-v2 (автоматически скачает ~2 ГБ при первом запуске)
tts = TTS(
    model_name="tts_models/multilingual/multi-dataset/xtts_v2",
    progress_bar=True
).to(device)

# Генерация с клонированием голоса
tts.tts_to_file(
    text="Я бля сегодня бля весь день бля работал и конкретно бля заебался. Всё, что я сейчас бля хочу - это набухаться бля и съесть бля огромного вкусного мамонта бля. Вот.",
    speaker_wav="audio/Vasya/audio_2026-01-30_17-49-34.ogg",          # 6–30 секунд твоей чистой речи (wav лучше всего)
    language="ru",                                   # русский поддерживается
    file_path="output_coqui.wav"
)

# Если хочешь послушать прямо в Jupyter
from IPython.display import Audio
Audio("output_coqui.wav", autoplay=True)

C:\Users\Shurik\anaconda3\envs\talking-head-assistant-280\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\Shurik\anaconda3\envs\talking-head-assistant-280\lib\site-packages\torchaudio\_backend\utils.py:213: UserWarning: In 2.9, this function's implementation will be changed to use torchaudio.load_with_torchcodec` under the hood. Some parameters like ``normalize``, ``format``, ``buffer_size``, and ``backend`` will be ignored. We recommend that you port your code to rely directly on TorchCodec's decoder instead: https://docs.pytorch.org/torchcodec/stable/generated/torchcodec.decoders.AudioDecoder.html#torchcodec.decoders.AudioDecoder.
  warnings.warn(


In [3]:
from huggingface_hub import snapshot_download

model_dir = snapshot_download(
    repo_id="NeuroDonu/RU-XTTS-DonuModel",
    local_dir="./models/RU-XTTS-DonuModel",
    local_dir_use_symlinks=False  # на Windows лучше без симлинков
)
print("Модель скачана в:", model_dir)

C:\Users\Shurik\anaconda3\envs\talking-head-assistant-280\lib\site-packages\huggingface_hub\utils\_validators.py:202: UserWarning: The `local_dir_use_symlinks` argument is deprecated and ignored in `snapshot_download`. Downloading to a local directory does not use symlinks anymore.
  warnings.warn(
Fetching 6 files: 100%|██████████████████████████████████████████████████████████████████| 6/6 [00:05<00:00,  1.00it/s]

Модель скачана в: E:\Coding\talking-head-assistant\models\RU-XTTS-DonuModel
